# FUZZY MATCHING
As with the previous file (01-FUZZYWUZZY-GIT.ipynb) I am matching up more misspelled names to the "Offical" name spelling in HR records.

In [ ]:
# import some modules
import os
import pandas as pd
import sqlite3
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Connecting to the database

In [ ]:
# dbconn = sqlite3.connect(':memory:')
dbconn = sqlite3.connect('pars.db')
print(dbconn)

Creating a cursor

In [ ]:
cursor = dbconn.cursor()
cursor

### Here are the IN files

In [ ]:
a_file = r"C:\Users\CHUGHES\Documents\PROJECTS\Python\001-PARS\DATA\HR-ECI-ALLEMP.csv"
# p_file = r"C:\Users\CHUGHES\Documents\PROJECTS\Python\001-PARS\DATA\2021-PAR-EDIT.csv"
file1 = r"C:\Users\CHUGHES\Documents\PAR Consolidation\2020\REWORK-DS\CSV exports\2019-Preliminary Assessment Review Database - February 2020-NO_PRA.csv"


## Working with PARS file
Naming the column headers I am interested in.

In [ ]:
p_header = {0:"NUMBER",1:"OPEN_DATE",2:"SOURCE", 3:"LNAME",4:"MNAME",5:"FNAME",6:"OFFICE",7:"CLOSED_DATE",8:"DET_COM",9:"INQUIRY",10:"WORKED_BY"}

In [ ]:
p_header

Selecting only the columns I want

In [ ]:
# df1 = pd.read_csv(file1, index_col=None)
df1 = pd.read_csv(file1, index_col=None, header=None, skiprows=1, usecols = [0,1,2,3,4,5,6,7,8,9,10])
df1.rename(columns = p_header, inplace=True)

In [ ]:
df1.head(5)

### Making sure the data is in UPPER Case

In [ ]:
df1.columns = map(str.upper, df1.columns) #  Make all the column headers upper case
df1['LNAME'] = df1['LNAME'].str.upper()
df1['FNAME'] = df1['FNAME'].str.upper()

In [ ]:
df1.head(5)

In [ ]:
len(df1.index) # Just testing that the number of rows stays the same.

In [ ]:
# df1['P_LNAME'] = df1.NAME_PACS.str.split(',', expand = True)[0] # Not Needed for this exercise
# df1['P_FNAME'] = df1.NAME_PACS.str.split(' ', expand = True)[2] # Not Needed for this exercise - splits first and last names

In [ ]:
df1.head(5)

In [ ]:
len(df1.index)  # Just testing that the number of rows stays the same.

In [ ]:
df1.dtypes

### Need to change data type to a string for the Lamda function to work
###  Also converting the string value of the Date fields to DateTime and 
### then to just a DATE
I just like the short dates, without all the zeros.  And I want YYYY-MM-DD format.

In [ ]:
df1 = df1.astype({"LNAME": str}, errors='raise') 
df1 = df1.astype({"FNAME": str}, errors='raise')
# df1['OPEN_DATE'].dt.date
df1['OPEN_DATE'] = pd.to_datetime(df1['OPEN_DATE']).dt.strftime('%Y-%m-%d')
df1['CLOSED_DATE'] = pd.to_datetime(df1['CLOSED_DATE']).dt.strftime('%Y-%m-%d')
df1['OPEN_DATE'] = pd.to_datetime(df1['OPEN_DATE'])
df1['CLOSED_DATE'] = pd.to_datetime(df1['CLOSED_DATE'])
# df1['OPEN_DATE'].dt.date
# df1['CLOSED_DATE'].dt.date
df1['OPEN_DATE'] = pd.to_datetime(df1['OPEN_DATE']).dt.date
df1['CLOSED_DATE'] = pd.to_datetime(df1['CLOSED_DATE']).dt.date
df1.dtypes

If this worked correctly, when inserted into SQLITE it will have the dates formated the way I want them.  AND the structure will indicate "DATE"  not DATETIME(64)

In [ ]:
df1.head(5)

### Joining the LAST_NAME and FIRST_NAME columns with a "," and a SPACE
### and adding the new column to the end

In [ ]:
df1['NAME'] = df1[['LNAME','FNAME']].apply(lambda x: ', '.join(x), axis=1)

### will fuzzymatch the "NAME" column with the Aplha Roster "A_NAME"
### column in the other file

In [ ]:
df1.head(10)

In [ ]:
len(df1.index)  # Just testing that the number of rows stays the same.

___

## Working the APLHA file (HR DATA)
I think the function below is identical to the first file (01-FUZZYWUZZY-GIT.ipynb)

In [ ]:
df2 = pd.read_csv(a_file)
df2.head(2)

In [ ]:
a_header = {0:"LOA",1:"A_NAME",3:"ECI"}

In [ ]:
a_header

In [ ]:
df2 = pd.read_csv(a_file, index_col=None, header=None, skiprows=1, usecols = [0,1,3])
df2.rename(columns = a_header, inplace=True)

In [ ]:
df2.head(10)

In [ ]:
dfunique = len(df2) - df2.nunique()
print (dfunique)

In [ ]:
# df2.drop_duplicates(subset=['NAME_ALPHA'],inplace=True)
# df2.drop_duplicates(subset=['NAME_PACS'])
#df2.head(5)

In [ ]:
df2 = df2.astype({"A_NAME": str}, errors='raise') 
# df1 = df1.astype({"FNAME": str}, errors='raise') 

In [ ]:
# df2['A_NAME'] = df2[['A_LNAME','A_FNAME']].apply(lambda x: ', '.join(x), axis=1)

In [ ]:
df2.head(5)

In [ ]:
df1.head(5)

___

In [ ]:
process.extractOne("HUGHES, A", df2['A_NAME'].to_list(), score_cutoff=90)

In [ ]:
process.extractOne("HUGHES, C", df1['NAME'].to_list(), score_cutoff=90)

### Below is the actual Fuzzy Match

In [ ]:
# df1['name_from_df2'] = df1['P_NAME'].apply(lambda x: process.extractOne(x, df2['A_NAME'].to_list(),score_cutoff=90))
# df2['PAR_NAME'] = df2['A_NAME'].apply(lambda x: process.extractOne(x, df1['NAME'].to_list(),score_cutoff=90))
df1['ALPHA_NAME'] = df1['NAME'].apply(lambda x: process.extractOne(x, df2['A_NAME'].to_list(),score_cutoff=90))
# df2.head(5)

In [ ]:
df1.head(5)

In [ ]:
temp_name_list = df1['ALPHA_NAME'].to_list()
temp_name_list = [_[0] if _ != None else None for _ in temp_name_list]
df1['ALPHA_NAME'] = temp_name_list

In [ ]:
df1.head(15)

In [ ]:
# df2.drop(columns=['PACR_NAME']) # Ooops

In [ ]:
len(df1.index)

___
### Put the dataframe into the database

In [ ]:
df1.to_sql('fuzz_19',con = dbconn, if_exists='replace', index=False)

___
### I used this query in SQLite to match things up
### where 'LOAAlpha is a table with all COMPANY employees
### with the LOA and ECI #'s

```
SELECT f.NUMBER,
       f.OPEN_DATE,
       f.SOURCE,
       l.LOA,
       l.ECI,
       f.LNAME,
       f.MNAME,
       f.FNAME,
       f.OFFICE,
       f.CLOSED_DATE,
       f.DET_COM,
       f.INQUIRY,
       f.WORKED_BY,
       f.NAME,
       f.ALPHA_NAME
  FROM fuzz_19 f
       LEFT JOIN
       LOAAlpha L ON f.ALPHA_NAME = l.NAME;

```
and that's it